<a href="https://colab.research.google.com/github/TasksAndreySorokin/KIA-GPT1/blob/feature%2Fupdate_faiss/Alex_Minkin/%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22KIAv3_%D1%81_%D0%BB%D0%BE%D0%BA%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_FAISS_%D0%B8_%D0%B0%D0%B2%D1%82%D0%BE%D0%B7%D0%B0%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B_%D0%B4%D0%BB%D1%8F_%D1%82%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

## Установка библиотек и функций

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8 openpyxl

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken
from io import BytesIO
import openpyxl
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import os
import platform

def clear_screen():
    system_platform = platform.system()  # определить операционную систему
    if system_platform == "Windows":
        os.system('cls')
    else:
        os.system('clear')

In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K # выбери нужную модель

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# функция для загрузки txt файла по ссылке
def load_text(url: str) -> str:
    # Download the document as plain text
    response = requests.get(url)
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# Функция загрузки базы знаний из файла и создания векторной базы
def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Router"),
        ("##", "Header2"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
    vectordateBase.save_local("/content/faiss")   # сохраняем векторную базу локально

    return vectordateBase



In [ ]:
# Функция загрузки векторной базы из папки
def load_vectordate_base(path_to_vectordate_base: str) -> str:

    vectordate_base = FAISS.load_local(path_to_vectordate_base, OpenAIEmbeddings())
    return vectordate_base

### Загрузка таблицы для тестирования

In [ ]:
# функция для загрузки таблицы по ссылке из гугл драйв
#def load_xls_pd(url: str) -> str:
def load_xls_pd(url: str, sheet_name: str = 0) -> str:

    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    if sheet_name is None:
        raise ValueError('Invalid sheet_name')

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    #data = pd.read_excel(BytesIO(response.content), sheet_name=0)  #чтение первого листа из книги
    data = pd.read_excel(BytesIO(response.content),sheet_name = sheet_name, header=None) #чтение конкретного листа из Книги Excel

    ''' Разъяснение:
        Когда мы хотим прочитать данные Excel с помощью функции pd.read_excel(), она требует передачи ей пути к файлу или объекта, представляющего файл.
        В параметре io функции pd.read_excel() необходимо указать путь к файлу (в виде строки, содержащей путь к файлу) или объект файлового типа (такой как BufferedWriter, BufferedReader и другие).
        response.content возвращает содержимое ответа на запрос HTTP в виде байтового массива (bytes array).
        Чтобы передать эти данные в функцию pd.read_excel(), нужно создать объект файла из байтового массива.
        Для этой цели используется объект BytesIO из модуля io, который предоставляет интерфейс для работы с данными в памяти, как если бы они находились в файле.
    '''
    return data

In [ ]:
# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1ewjENXCLwfmzGA9FFTqb6lcg4Ob9N505Lg-4H8oQvoI"
sheet_name='Вопросы с 10.11'

#data = load_xls_pd(google_sheet_url)   #если в таблице только один лист
df = load_xls_pd(google_sheet_url, sheet_name)

In [ ]:
# проверка
df

In [ ]:
# установить первую строку в виде строки заголовков столбцов
df.set_axis(df.iloc[0], axis='columns', inplace=True)
# проверка
df.info()
df

In [ ]:
#df['Чанки'] = pd.Series(dtype=str)   # так можно добавить столбец "Чанки", если его нет

In [ ]:
#df = df.drop('Чанки', axis=1)         # а так можно удалить столбец

In [ ]:
#df.insert(loc=10, column='Чанки', value='') #2ой способ добавить пустой столбец "Чанки", если он отсутствует 11 столбцом

## Загружаем векторное хранилище либо из базы знаний:
Предварительно нужно загрузить файл базы знаний 'database.md' в колаб

In [ ]:
#Если заново собирать векторную базу из базы знаний, то нужно раскоментировать этот код
'''
!git clone -b develop https://github.com/TasksAndreySorokin/KIA-GPT1.git
!cp /content/KIA-GPT1/knowledge/database.md /content/database.md
!rm -r /content/KIA-GPT1
knowledge_base = load_file_knowledge('database.md')
'''

In [ ]:
#!zip -r faiss.zip faiss     #для архивирования папки, чтобы ее можно было скачать себе

## Либо используем уже готовую векторную базу (два раза НЕ НУЖНО):
Предварительно загрузим папку FAISS_routing с векторными базами в колаб и скопируем ее в нужное место:

In [ ]:
!git clone -b develop https://github.com/TasksAndreySorokin/KIA-GPT1.git
!cp -r /content/KIA-GPT1/knowledge/faiss /content/faiss
!rm -r /content/KIA-GPT1

In [ ]:
#!unzip /content/faiss.zip   #можно загрузить в колаб из локальной  папки с векторным хранилищем и разархиваровать

Теперь подгружаем векторные хранилища:

In [ ]:
knowledge_base = load_vectordate_base('faiss')

In [ ]:
# функция суммаризации диалогов
def _summarize_topic(dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content

## Функция запроса к ChatGPT

In [ ]:
def answer_index(topic, temp=0.1, top_similar_documents=3):
        summarize_text = ''
        HISTORY = '' # ОТКЛЮЧАЕТ САММОРИЗАЦИЮ! ЕСТЬ НУЖНО ЕЕ ВКЛЮЧИТЬ ТОГДА ЗАКОМЕНТИРУЙ
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
            print(f'САММАРИ \n=== {summarize_text} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)

        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f' ___ score = {score} ___ Metadata документа = {doc.metadata}\n___Отрывок документа №{i + 1}___\n{content}\n'
                # раскомментируй строки ниже для вывода на экран найденных чанков
                '''
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f'\n{content}\n')
                '''
                responses.append(response)

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа клиенту: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        if len(HISTORY) > 0:      #а нужно ли здесь проверять? Ведь она у нас изначально пустая, т.е. ничего не добавиться
            HISTORY.append((topic, answer if answer is not None else ''))

        return  insert_newlines(answer), responses

In [ ]:
# Промпт моделей машин
prompt = load_text('https://github.com/TasksAndreySorokin/KIA-GPT1/blob/develop/Prompts/kia_main_%20promt.txt')

## Для обработки диалогов вручную

In [ ]:

def run_dialog():

    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_index(user_question)
        print('\nМенеджер: ', answer+'\n\n')

    return


In [ ]:
verbose=0
#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog()

## Сохранение ответов ChatGPT и чанков в таблицу

In [ ]:
# Переменная для подсчета количества строк, обработанных по условию
count = 0
# Цикл для обхода строк DataFrame
for index, row in tqdm(df.iterrows()):
    if (row['Тема'] == 'Технологии') & (pd.isna(row['Ответ GPT'])):       # Выбираем строки со своей Темой для тестирования, и если не пустые
        # Ваш код для обработки данных текущей строки
        user_question = row['Вопрос']
        answer, chanks = answer_index(user_question)

        # Записываем результат в столбец "Ответ"
        df.at[index, 'Ответ GPT'] = answer
        # Записываем чанки
        df.at[index, 'Чанки'] = chanks
        # Увеличиваем значение счетчика и проверяем, достигли ли мы нужного количества запросов
        count += 1
        print('Количество обработанных запросов: ' + count, end=' ')
        if count >= 5:          # количество задаваемых вопросов к ChatGPT
            break


In [ ]:
df[df['Тема'] == 'Технологии'][['Вопрос', 'Ответ GPT', 'Чанки']].head(15)   #проверка записей в таблице

In [ ]:
df[(df['Тема'] == 'Технологии') & (df['Ответ GPT'].isna())].head(10) #проверка записей в таблице

In [ ]:
df.to_excel('test.xlsx')      # загрузка df в таблицу 'test.xlsx'

Далее:
- в исходной таблице установить фильтр для отображения только своего раздела, чтобы не перезаписать чужие разделы.
- открыть созданную таблицу в Гугл Таблицах, установить фильтр для отображения только своего раздела, чтобы не перезаписать чужие разделы.
- скопировать по очереди целиком столбцы 'Ответ GPT' и 'Чанки' в исходную таблицу для тестирования.